<a href="https://colab.research.google.com/github/mazhar75/Natural-Language-Processing/blob/main/making_input_for_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00


In [2]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
df=pd.read_csv('dataset.csv')

In [4]:
df.columns = df.columns.str.replace(r'\s+', '_', regex=True)

In [5]:
df.columns=df.columns.str.lower()

In [10]:
for x in df.columns:
    df[x]=df[x].str.lower()
df['emotion'][100]

'happy'

In [11]:
dataset=Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'emotion'],
    num_rows: 232
})

In [12]:
emotion_map = {
    'happy': 0,
    'sad': 1,
    'surprise': 2,
    'anger': 3,
    'fear': 4,
    'neutral': 5
}
dataset = dataset.map(lambda example: {"emotion": emotion_map[example["emotion"]]})

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

In [13]:
dataset['emotion'][100]

0

In [14]:
dataset = dataset.train_test_split(test_size=0.2)  # 80% train, 20% test
dataset["validation"] = dataset["test"].train_test_split(test_size=0.5)["train"]  # 10% validation, 10% test

In [15]:
dataset_dict = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["validation"],
    "test": dataset["test"]
})


In [16]:
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['text', 'emotion'],
        num_rows: 185
    })
    validation: Dataset({
        features: ['text', 'emotion'],
        num_rows: 23
    })
    test: Dataset({
        features: ['text', 'emotion'],
        num_rows: 47
    })
})


In [17]:
from transformers import AutoTokenizer
checkpoint = "bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
def tokenize_function(example):
    return tokenizer(example["text"],truncation=True)

In [19]:
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 185
    })
    validation: Dataset({
        features: ['text', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 23
    })
    test: Dataset({
        features: ['text', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 47
    })
})

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
tokenized_datasets["train"][0]
tokenized_datasets["train"].column_names

['text', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask']

In [22]:
tokenized_datasets=tokenized_datasets.remove_columns(['text'])

In [23]:
tokenized_datasets=tokenized_datasets.rename_column('emotion','labels')

In [24]:
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [25]:
tokenized_datasets.set_format("torch")

In [26]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [27]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 23]),
 'token_type_ids': torch.Size([8, 23]),
 'attention_mask': torch.Size([8, 23])}

In [28]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.8151, grad_fn=<NllLossBackward0>) torch.Size([8, 6])


In [30]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

72


In [32]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [33]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/72 [00:00<?, ?it/s]

In [34]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [35]:
import evaluate

metric = evaluate.load("glue", "mrpc")
# Load the metric, but specify the 'average' parameter for multi-class
metric = evaluate.load("accuracy")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Compute the metric with the appropriate averaging method for accuracy
results = metric.compute()
print(results)

{'accuracy': 0.30434782608695654}


**Creating Pipeline**

In [36]:
from huggingface_hub import notebook_login

notebook_login()

In [39]:
model.push_to_hub("Sylheti_Banglish_Text2Emotion")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mazhar75/Sylheti_Banglish_Text2Emotion/commit/947ff19635b23fc6da30bbd601216116a5c41721', commit_message='Upload BertForSequenceClassification', commit_description='', oid='947ff19635b23fc6da30bbd601216116a5c41721', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mazhar75/Sylheti_Banglish_Text2Emotion', endpoint='https://huggingface.co', repo_type='model', repo_id='mazhar75/Sylheti_Banglish_Text2Emotion'), pr_revision=None, pr_num=None)

In [40]:
tokenizer.push_to_hub("Sylheti_Banglish_Text2Emotion")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mazhar75/Sylheti_Banglish_Text2Emotion/commit/a42b6cac233fbe6aaba75542bf7708dc331ffa26', commit_message='Upload tokenizer', commit_description='', oid='a42b6cac233fbe6aaba75542bf7708dc331ffa26', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mazhar75/Sylheti_Banglish_Text2Emotion', endpoint='https://huggingface.co', repo_type='model', repo_id='mazhar75/Sylheti_Banglish_Text2Emotion'), pr_revision=None, pr_num=None)

In [41]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="mazhar75/Sylheti_Banglish_Text2Emotion")

config.json:   0%|          | 0.00/995 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


**Using My Own PipeLine : Yaaay !**

In [42]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mazhar75/Sylheti_Banglish_Text2Emotion")
model = AutoModelForSequenceClassification.from_pretrained("mazhar75/Sylheti_Banglish_Text2Emotion")

In [47]:
text="aijku amr mon vala nay ba"
tokenized_text=tokenizer(text,return_tensors='pt')
predictions=model(**tokenized_text)
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.3444,  0.6071, -0.4316,  0.5353, -0.7564, -0.1746]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)